In [26]:
import pickle
# Open the file for reading
with open('links_completed.data', 'rb') as fp:
        links = pickle.load(fp)

In [44]:
# Group most 10 frequent
#link = {"Title":"","Url":"","Summary":"","Keywords":""};
group = {"Group_Name":"", "Group_Total":"", "Group_Links":[] , "Group_Scores":[]}
groups = []

groups_name = []
groups_total = []
for link in links:
    for keyword in link["Keywords"]:
        if keyword not in groups_name:
            groups_name.append(keyword)
            groups_total.append(1)
        else:
            groups_total[groups_name.index(keyword)] += 1
    
topten = sorted(zip(groups_total, groups_name), reverse=True)[:10]
print(topten)

for g in topten:
    group["Group_Total"] = g[0]
    group["Group_Name"] = g[1]
    for link in links:
        for keyword in link["Keywords"]:
            if keyword == g[1]:
                group["Group_Links"].append(link["Url"])
                break
    groups.append(group.copy())
    
print(groups[0])

[(40, 'loop'), (8, 'tutorial'), (7, 'programming'), (6, 'statement'), (6, 'program'), (6, 'microsoft'), (6, 'condition'), (6, 'code'), (5, 'team'), (4, 'technology')]
{'Group_Name': 'loop', 'Group_Total': 40, 'Group_Links': ['https://www.programiz.com/c-programming/c-for-loop', 'https://www.geeksforgeeks.org/c-loops/', 'https://www.guru99.com/c-loop-statement.html', 'https://www.thoughtco.com/definition-of-loop-958105', 'https://www.geeksforgeeks.org/cpp-loops/', 'https://www.educba.com/loops-in-c-plus-plus/', 'https://www.programiz.com/cpp-programming/for-loop', 'https://history-computer.com/understanding-loops-in-c-with-examples/', 'https://www.freecodecamp.org/news/for-loops-in-c/', 'https://www.tutorialspoint.com/cprogramming/c_loops.htm', 'https://www.codecademy.com/resources/docs/c/loops', 'https://www.freecodecamp.org/news/for-loops-in-c/', 'https://www.techtarget.com/whatis/definition/loop', 'https://overiq.com/c-programming-101/the-infinite-loop-in-c/', 'https://www.geeksforge

In [18]:
from gensim.models import Word2Vec
from sentence_transformers.util import cos_sim
from sentence_transformers import SentenceTransformer, util
import numpy as np

word_model = Word2Vec.load("word_model.model")

sents_model = SentenceTransformer('all-MiniLM-L6-v2')
sents_embeddings = np.load("sents_embedding.npy")

processed_query="what is loop"

In [42]:
import yake

# ranking by weight
def cal_similar (processed_query, group_name, summary, link_keywords):
    #extract keyword from user query
    kw_extractor = yake.KeywordExtractor()
    language = "en"
    max_ngram_size = 1
    deduplication_threshold = 0.9
    numOfKeywords = 1
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, 
                                                dedupLim=deduplication_threshold, 
                                                top=numOfKeywords, features=None)
    user_keywords = custom_kw_extractor.extract_keywords(processed_query)
    keyword = user_keywords[0][0]
    
    #calculate word similarity score
    word_score = 0
    for link_keyword in link_keywords:
        #word_score += model.wv.similarity(string,string)
        try:
            word_score += word_model.wv.similarity(keyword,link_keyword)
        except:
            word_score += 0
        try:
            word_score += word_model.wv.similarity(group_name,link_keyword)
        except:
            word_score += 0
    
    #calculate sentence similarity score
    sents_score = 0
    #sents_model.encode(['string'])
    query_embedding1 = sents_model.encode([processed_query])
    for sentence in summary:
        query_embedding2 = sents_model.encode([str(sentence)])
        sents_score += cos_sim(query_embedding1, query_embedding2)
        
    #calulate total similarity score
    total_score = 60/100*word_score + 40/100*sents_score
    return float(total_score)

for group in groups:
    temp_links = []
    temp_score = []
    temp_links = group["Group_Links"]
    for temp_link in temp_links:
        for link in links:
            if temp_link == link["Url"]:
                summary = link["Summary"]
                link_keywords = link["Keywords"]
        sim_score =  cal_similar (processed_query, group["Group_Name"], summary, link_keywords)
        temp_score.append(sim_score)
    temp = sorted(zip(temp_links, temp_score), reverse=True)
    print(temp)
    temp_links = [t[0] for t in temp]
    temp_score = [t[1] for t in temp]
    group["Group_Links"] = temp_links
    group["Group_Scores"] = temp_score

[('https://www.w3schools.in/csharp/loops', 1.8148722648620605), ('https://www.w3schools.in/c-programming/loops', 1.8696010112762451), ('https://www.w3schools.com/cs/cs_while_loop.php', 0.48994266986846924), ('https://www.w3schools.com/cs/cs_while_loop.php', 0.48994266986846924), ('https://www.w3schools.com/c/c_for_loop.php', 0.6570769548416138), ('https://www.w3schools.com/c/c_for_loop.php', 0.6570769548416138), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 3.1080305576324463), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 3.1080305576324463), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 2.455548048019409), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 2.455548048019409), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 3.1459758281707764), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 3.1459758281707764), ('https://www.tutorialspoint.com/cprogramming/c_for_loop.htm', 2.664804458618164), ('https:/

[('https://www.w3schools.in/csharp/loops', 1.9195337295532227), ('https://www.w3schools.in/c-programming/loops', 2.2138452529907227), ('https://www.w3schools.com/cs/cs_while_loop.php', 0.821736216545105), ('https://www.w3schools.com/cs/cs_while_loop.php', 0.821736216545105), ('https://www.w3schools.com/c/c_for_loop.php', 0.9888705015182495), ('https://www.w3schools.com/c/c_for_loop.php', 0.9888705015182495), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 2.0723607540130615), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 2.0723607540130615), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 2.236851930618286), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 2.236851930618286), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 2.3480801582336426), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 2.3480801582336426), ('https://www.tutorialspoint.com/cprogramming/c_for_loop.htm', 2.109571933746338), ('https://www

[('https://www.w3schools.in/csharp/loops', 1.7047710418701172), ('https://www.w3schools.in/c-programming/loops', 1.7352619171142578), ('https://www.w3schools.com/cs/cs_while_loop.php', 1.1193132400512695), ('https://www.w3schools.com/cs/cs_while_loop.php', 1.1193132400512695), ('https://www.w3schools.com/c/c_for_loop.php', 1.286447525024414), ('https://www.w3schools.com/c/c_for_loop.php', 1.286447525024414), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 2.1730451583862305), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 2.1730451583862305), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 1.7943896055221558), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 1.7943896055221558), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 2.449409246444702), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 2.449409246444702), ('https://www.tutorialspoint.com/cprogramming/c_for_loop.htm', 2.1947121620178223), ('https://ww

[('https://www.w3schools.in/csharp/loops', 1.6166977882385254), ('https://www.w3schools.in/c-programming/loops', 1.7765605449676514), ('https://www.w3schools.com/cs/cs_while_loop.php', 0.46539103984832764), ('https://www.w3schools.com/cs/cs_while_loop.php', 0.46539103984832764), ('https://www.w3schools.com/c/c_for_loop.php', 0.6325253248214722), ('https://www.w3schools.com/c/c_for_loop.php', 0.6325253248214722), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 2.8036580085754395), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 2.8036580085754395), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 2.2712526321411133), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 2.2712526321411133), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 2.792447328567505), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 2.792447328567505), ('https://www.tutorialspoint.com/cprogramming/c_for_loop.htm', 2.4076271057128906), ('https:

[('https://www.w3schools.in/csharp/loops', 1.7820907831192017), ('https://www.w3schools.in/c-programming/loops', 1.9231908321380615), ('https://www.w3schools.com/cs/cs_while_loop.php', 1.1142178773880005), ('https://www.w3schools.com/cs/cs_while_loop.php', 1.1142178773880005), ('https://www.w3schools.com/c/c_for_loop.php', 1.281352162361145), ('https://www.w3schools.com/c/c_for_loop.php', 1.281352162361145), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 1.9577805995941162), ('https://www.tutorialspoint.com/csharp/csharp_loops.htm', 1.9577805995941162), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 1.7913932800292969), ('https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 1.7913932800292969), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 2.2328171730041504), ('https://www.tutorialspoint.com/cprogramming/c_loops.htm', 2.2328171730041504), ('https://www.tutorialspoint.com/cprogramming/c_for_loop.htm', 2.0473055839538574), ('https://

In [43]:
print(groups[1])

{'Group_Name': 'tutorial', 'Group_Total': 8, 'Group_Links': ['https://www.w3schools.in/csharp/loops', 'https://www.w3schools.in/c-programming/loops', 'https://www.w3schools.com/cs/cs_while_loop.php', 'https://www.w3schools.com/cs/cs_while_loop.php', 'https://www.w3schools.com/c/c_for_loop.php', 'https://www.w3schools.com/c/c_for_loop.php', 'https://www.tutorialspoint.com/csharp/csharp_loops.htm', 'https://www.tutorialspoint.com/csharp/csharp_loops.htm', 'https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 'https://www.tutorialspoint.com/cprogramming/c_while_loop.htm', 'https://www.tutorialspoint.com/cprogramming/c_loops.htm', 'https://www.tutorialspoint.com/cprogramming/c_loops.htm', 'https://www.tutorialspoint.com/cprogramming/c_for_loop.htm', 'https://www.tutorialspoint.com/cprogramming/c_do_while_loop.htm', 'https://www.tutorialspoint.com/cprogramming/c_do_while_loop.htm', 'https://www.thoughtco.com/definition-of-loop-958105', 'https://www.thoughtco.com/definition-of-loop

In [50]:
i = 1
chosen_group = groups[i]
print(chosen_group["Group_Name"] + ": ")
for group_ele in chosen_group["Group_Links"]:
    for link in links:
        if group_ele == link["Url"]:
            print(link["Title"] + " " + link ["Url"])

tutorial: 
C for Loop (With Examples) - Programiz https://www.programiz.com/c-programming/c-for-loop
C - Loops - GeeksforGeeks https://www.geeksforgeeks.org/c-loops/
Loops in C: For, While, Do While looping Statements [Examples] - Guru99 https://www.guru99.com/c-loop-statement.html
What Are Loops in Computer Programs? - ThoughtCo https://www.thoughtco.com/definition-of-loop-958105
C++ Loops - GeeksforGeeks https://www.geeksforgeeks.org/cpp-loops/
Loops in C++ | Different Types of Loops in C++ with Examples - EDUCBA https://www.educba.com/loops-in-c-plus-plus/
C++ for Loop (With Examples) - Programiz https://www.programiz.com/cpp-programming/for-loop
Understanding Loops In C, With Examples - History-Computer https://history-computer.com/understanding-loops-in-c-with-examples/
For Loops in C - Explained with Code Examples - freeCodeCamp.org https://www.freecodecamp.org/news/for-loops-in-c/
For Loops in C - Explained with Code Examples - freeCodeCamp.org https://www.freecodecamp.org/news/